# Importamos datos

In [82]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import category_encoders as ce # Para Target Encoding si se usa
import itertools # Para generar combinaciones de características
import warnings

# Ignorar warnings para mantener la salida limpia
warnings.filterwarnings('ignore')

# Cargar datos
print("Cargando datos train.csv y test.csv...")
try:
    df_entrenamiento_bruto = pd.read_csv('train.csv')
    df_prueba_bruto = pd.read_csv('test.csv')
    print("Datos cargados correctamente.")
    print("\nPrimeras 5 filas de df_entrenamiento_bruto:")
    print(df_entrenamiento_bruto.head())
except FileNotFoundError:
    print("ERROR: Asegúrate de que 'train.csv' y 'test.csv' estén en el directorio correcto.")
    df_entrenamiento_bruto = pd.DataFrame() 
    df_prueba_bruto = pd.DataFrame()

Cargando datos train.csv y test.csv...
Datos cargados correctamente.

Primeras 5 filas de df_entrenamiento_bruto:
      id            timestamp modo_operacion operador  temperatura    presion  \
0  15000  2021-04-24 03:00:00  mantenimiento      NaN    20.428941  58.474497   
1  15001  2020-09-26 19:00:00  mantenimiento        E     1.332931  84.784184   
2  15002  2021-07-02 15:00:00           auto        D    17.501579  65.057682   
3  15003  2020-02-05 22:00:00           auto      NaN     8.118801  70.127615   
4  15004  2020-09-05 18:00:00         manual      NaN    -0.353122  90.738090   

   sensor_ruido   sensor_3  fallo  
0      0.349400  18.301299      0  
1     -2.252684   7.295245      0  
2      0.359853  34.271305      0  
3      0.489166  -3.748938      0  
4     -1.505866  10.157637      0  


In [54]:
objetivo = 'fallo'

columnas_numericas_base = ['temperatura', 'presion', 'sensor_ruido', 'sensor_3']
columnas_categoricas = ['modo_operacion', 'operador'] 
columna_id = 'id' # Estan aparte para poder tratarlas ahora
columna_timestamp = 'timestamp'

print("--- Variables Globales Definidas ---")
print(f"  Objetivo: {objetivo}")
print(f"  Características numéricas base: {columnas_numericas_base}")
print(f"  Características categóricas: {columnas_categoricas}")
print(f"  Columna ID a eliminar: {columna_id}")
print(f"  Columna Timestamp a procesar: {columna_timestamp}")

# Mostrar NaNs
print("\nNaNs en df_entrenamiento_bruto:")
print(df_entrenamiento_bruto.isnull().sum())
print("\nNaNs en df_prueba_bruto:")
print(df_prueba_bruto.isnull().sum())

--- Variables Globales Definidas ---
  Objetivo: fallo
  Características numéricas base: ['temperatura', 'presion', 'sensor_ruido', 'sensor_3']
  Características categóricas: ['modo_operacion', 'operador']
  Columna ID a eliminar: id
  Columna Timestamp a procesar: timestamp

NaNs en df_entrenamiento_bruto:
id                   0
timestamp            0
modo_operacion       0
operador          2731
temperatura          0
presion            625
sensor_ruido         0
sensor_3             0
fallo                0
dtype: int64

NaNs en df_prueba_bruto:
id                   0
timestamp            0
modo_operacion       0
operador          2700
temperatura          0
presion            625
sensor_ruido         0
sensor_3             0
dtype: int64


# Preparamos Datos

## Columnas id y TimeStamp

In [55]:
# Crear copias de trabajo
df_entrenamiento = df_entrenamiento_bruto.copy()
df_prueba = df_prueba_bruto.copy()

print(f"  Eliminando '{columna_id}' de df_entrenamiento.")
df_entrenamiento = df_entrenamiento.drop(columns=[columna_id])
print(f"  Eliminando '{columna_id}' de df_prueba.")
df_prueba = df_prueba.drop(columns=[columna_id])


def dividirTimeStamp(df, nombre_columna):    
    df[nombre_columna] = pd.to_datetime(df[nombre_columna])

    df['hora'] = df[nombre_columna].dt.hour
    df['dia_semana'] = df[nombre_columna].dt.dayofweek # Lunes=0, Domingo=6
    df['mes'] = df[nombre_columna].dt.month
    df['dia_del_anio'] = df[nombre_columna].dt.dayofyear
    df['es_fin_de_semana'] = (df[nombre_columna].dt.dayofweek >= 5).astype(int) 
    
    caracteristicas_generadas = ['hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']
    
    # Se elimina la columna del df original
    df = df.drop(columns=[nombre_columna])
    return df, caracteristicas_generadas

df_entrenamiento, caracteristicas_tiempo_generadas_train = dividirTimeStamp(df_entrenamiento, columna_timestamp)
df_prueba, _ = dividirTimeStamp(df_prueba, columna_timestamp) # No necesitamos la lista de test, asumimos que es la misma

todas_columnas_numericas = columnas_numericas_base + caracteristicas_tiempo_generadas_train

print(f"\nNuevas características de tiempo generadas: {caracteristicas_tiempo_generadas_train}")
print(f"Todas las características numéricas para imputación serán: {todas_columnas_numericas}")
print(df_entrenamiento.head())


  Eliminando 'id' de df_entrenamiento.
  Eliminando 'id' de df_prueba.

Nuevas características de tiempo generadas: ['hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']
Todas las características numéricas para imputación serán: ['temperatura', 'presion', 'sensor_ruido', 'sensor_3', 'hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']
  modo_operacion operador  temperatura    presion  sensor_ruido   sensor_3  \
0  mantenimiento      NaN    20.428941  58.474497      0.349400  18.301299   
1  mantenimiento        E     1.332931  84.784184     -2.252684   7.295245   
2           auto        D    17.501579  65.057682      0.359853  34.271305   
3           auto      NaN     8.118801  70.127615      0.489166  -3.748938   
4         manual      NaN    -0.353122  90.738090     -1.505866  10.157637   

   fallo  hora  dia_semana  mes  dia_del_anio  es_fin_de_semana  
0      0     3           5    4           114                 1  
1      0    19           5    9       

## Quitar los Nan

### En las Numericas
Hay dos opciones: 

Imputar con la Mediana, se entrena a un modelo y se sustituye los Nan con la prediccion. Es buena pero no siempre fiable

Imputar con KNNImputer, una version mas avanzada de imputar con la mediana. El problema es que hay que escalar los datos antes de pasarlos por el Imputer

In [56]:
df_entrenamiento_procesado = df_entrenamiento.copy()
df_prueba_procesado = df_prueba.copy()

# Columnas numéricas existentes en los dataframes para imputar
columnas_numericas_a_imputar_train = [col for col in todas_columnas_numericas if col in df_entrenamiento_procesado.columns]
columnas_numericas_a_imputar_test = [col for col in todas_columnas_numericas if col in df_prueba_procesado.columns]

In [57]:
# Imputacion de la mediana ==> Peor || Comentado para que no ejecute
"""
imputador_mediana = SimpleImputer(strategy='median')
df_entrenamiento_procesado[columnas_numericas_a_imputar_train] = imputador_mediana.fit_transform(df_entrenamiento_procesado[columnas_numericas_a_imputar_train])

cols_comunes_test = [col for col in columnas_numericas_a_imputar_test if col in imputador_mediana.feature_names_in_]
df_prueba_procesado[cols_comunes_test] = imputador_mediana.transform(df_prueba_procesado[cols_comunes_test])
"""

"\nimputador_mediana = SimpleImputer(strategy='median')\ndf_entrenamiento_procesado[columnas_numericas_a_imputar_train] = imputador_mediana.fit_transform(df_entrenamiento_procesado[columnas_numericas_a_imputar_train])\n\ncols_comunes_test = [col for col in columnas_numericas_a_imputar_test if col in imputador_mediana.feature_names_in_]\ndf_prueba_procesado[cols_comunes_test] = imputador_mediana.transform(df_prueba_procesado[cols_comunes_test])\n"

In [58]:
# KNNImputer ==> La que mejor va
escalador_knn = StandardScaler() 

# Para poder usar KNNImputer, hace falta escalar los datos
df_entrenamiento_escalado_num = escalador_knn.fit_transform(df_entrenamiento_procesado[columnas_numericas_a_imputar_train])

imputador_knn = KNNImputer(n_neighbors=5) 
df_entrenamiento_imputado_escalado = imputador_knn.fit_transform(df_entrenamiento_escalado_num) # Fit y transform en train escalado

# Desescalar train
df_entrenamiento_procesado[columnas_numericas_a_imputar_train] = escalador_knn.inverse_transform(df_entrenamiento_imputado_escalado)

# Transformar test
cols_comunes_test_knn = [col for col in columnas_numericas_a_imputar_test if col in escalador_knn.feature_names_in_]
df_prueba_escalado_num = escalador_knn.transform(df_prueba_procesado[cols_comunes_test_knn])
df_prueba_imputado_escalado = imputador_knn.transform(df_prueba_escalado_num) # Solo transform en test escalado
df_prueba_procesado[cols_comunes_test_knn] = escalador_knn.inverse_transform(df_prueba_imputado_escalado)

### En las categóricas
Aqui simplemente reemplazamos los Nan por un nuevo valor llamado "Desconocido"

In [59]:
columnas_categoricas_a_imputar_train = [col for col in columnas_categoricas if col in df_entrenamiento_procesado.columns]
columnas_categoricas_a_imputar_test = [col for col in columnas_categoricas if col in df_prueba_procesado.columns]

imputador_constante_cat = SimpleImputer(strategy='constant', fill_value='Faltante') # Traducido "Missing"

# Ajustar y transformar train
df_entrenamiento_procesado[columnas_categoricas_a_imputar_train] = imputador_constante_cat.fit_transform(df_entrenamiento_procesado[columnas_categoricas_a_imputar_train])

# Transformar test
cols_comunes_test_cat = [col for col in columnas_categoricas_a_imputar_test if col in imputador_constante_cat.feature_names_in_]
df_prueba_procesado[cols_comunes_test_cat] = imputador_constante_cat.transform(df_prueba_procesado[cols_comunes_test_cat])

### Comprobacion de los Nans

In [60]:
print("\nComprobación de NaNs después de toda la imputación:")
print("NaNs en df_entrenamiento_procesado (solo columnas con NaNs):")
print(df_entrenamiento_procesado.isnull().sum().loc[lambda x: x > 0])
print("\nNaNs en df_prueba_procesado (solo columnas con NaNs):")
print(df_prueba_procesado.isnull().sum().loc[lambda x: x > 0])


Comprobación de NaNs después de toda la imputación:
NaNs en df_entrenamiento_procesado (solo columnas con NaNs):
Series([], dtype: int64)

NaNs en df_prueba_procesado (solo columnas con NaNs):
Series([], dtype: int64)


## Tratar las categoricas para los modelos
Vamos a probar dos, OneHot y Target Encoding.

In [61]:
# df_entrenamiento_final y df_prueba_final serán el resultado de esta celda
df_entrenamiento_final = df_entrenamiento_procesado.copy()
df_prueba_final = df_prueba_procesado.copy()

# Columnas categóricas existentes para la codificación
columnas_categoricas_a_codificar_train = [col for col in columnas_categoricas if col in df_entrenamiento_final.columns]
columnas_categoricas_a_codificar_test = [col for col in columnas_categoricas if col in df_prueba_final.columns]

In [62]:
# OneHot ==> Peor || Comentado para que no ejecute
"""
codificador_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

# Ajustar y transformar train
matriz_codificada_entrenamiento = codificador_ohe.fit_transform(df_entrenamiento_final[columnas_categoricas_a_codificar_train])
df_codificado_entrenamiento = pd.DataFrame(matriz_codificada_entrenamiento, 
                                            columns=codificador_ohe.get_feature_names_out(columnas_categoricas_a_codificar_train), 
                                            index=df_entrenamiento_final.index)

# Eliminar categóricas originales y unir las codificadas
df_entrenamiento_final = df_entrenamiento_final.drop(columns=columnas_categoricas_a_codificar_train)
df_entrenamiento_final = df_entrenamiento_final.join(df_codificado_entrenamiento)


matriz_codificada_prueba = codificador_ohe.transform(df_prueba_final[columnas_categoricas_a_codificar_test])
df_codificado_prueba = pd.DataFrame(matriz_codificada_prueba, 
                                    columns=codificador_ohe.get_feature_names_out(columnas_categoricas_a_codificar_test), 
                                    index=df_prueba_final.index)
df_prueba_final = df_prueba_final.drop(columns=columnas_categoricas_a_codificar_test)
df_prueba_final = df_prueba_final.join(df_codificado_prueba)
"""


"\ncodificador_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')\n\n# Ajustar y transformar train\nmatriz_codificada_entrenamiento = codificador_ohe.fit_transform(df_entrenamiento_final[columnas_categoricas_a_codificar_train])\ndf_codificado_entrenamiento = pd.DataFrame(matriz_codificada_entrenamiento, \n                                            columns=codificador_ohe.get_feature_names_out(columnas_categoricas_a_codificar_train), \n                                            index=df_entrenamiento_final.index)\n\n# Eliminar categóricas originales y unir las codificadas\ndf_entrenamiento_final = df_entrenamiento_final.drop(columns=columnas_categoricas_a_codificar_train)\ndf_entrenamiento_final = df_entrenamiento_final.join(df_codificado_entrenamiento)\n\n\nmatriz_codificada_prueba = codificador_ohe.transform(df_prueba_final[columnas_categoricas_a_codificar_test])\ndf_codificado_prueba = pd.DataFrame(matriz_codificada_prueba, \n                           

In [63]:
# Target encoder ==> Mejor
codificador_objetivo = ce.TargetEncoder(cols=columnas_categoricas_a_codificar_train, smoothing=10.0)

# Ajustar y transformar train (modifica el DataFrame directamente)
df_entrenamiento_final[columnas_categoricas_a_codificar_train] = codificador_objetivo.fit_transform(df_entrenamiento_final[columnas_categoricas_a_codificar_train], df_entrenamiento_final[objetivo])

# Transformar test
df_prueba_final[columnas_categoricas_a_codificar_test] = codificador_objetivo.transform(df_prueba_final[columnas_categoricas_a_codificar_test])

In [64]:
print(f"\nForma de df_entrenamiento_final (después de codificación categórica): {df_entrenamiento_final.shape if 'df_entrenamiento_final' in locals() else 'No definido'}")
if 'df_entrenamiento_final' in locals() and not df_entrenamiento_final.empty:
    print("Tipos de datos en df_entrenamiento_final:")
    print(df_entrenamiento_final.dtypes)
    print("\nVerificación final de NaNs en df_entrenamiento_final (deberían ser 0):")
    print(df_entrenamiento_final.isnull().sum().loc[lambda x: x > 0])


Forma de df_entrenamiento_final (después de codificación categórica): (15000, 12)
Tipos de datos en df_entrenamiento_final:
modo_operacion      float64
operador            float64
temperatura         float64
presion             float64
sensor_ruido        float64
sensor_3            float64
fallo                 int64
hora                float64
dia_semana          float64
mes                 float64
dia_del_anio        float64
es_fin_de_semana    float64
dtype: object

Verificación final de NaNs en df_entrenamiento_final (deberían ser 0):
Series([], dtype: int64)


## Datos sin Tocar

In [65]:
df_entrenamiento_base = df_entrenamiento_final.copy()
df_prueba_base = df_prueba_final.copy()

## Datos quitando las no prometedoras (Sensor Ruido)

In [66]:
df_entrenamiento_reducido = df_entrenamiento_final.copy()
df_prueba_reducido = df_prueba_final.copy()
columna_a_quitar = 'sensor_ruido' # Es la unica que no aumenta la informacion, o la que parece al menos que no

df_entrenamiento_reducido = df_entrenamiento_reducido.drop(columns=[columna_a_quitar])
df_prueba_reducido = df_prueba_reducido.drop(columns=[columna_a_quitar])

## Datos relacionados TODOS con TODOS

In [67]:
df_entrenamiento_interacciones = df_entrenamiento_final.copy()
df_prueba_interacciones = df_prueba_final.copy()


for col1, col2 in itertools.combinations(columnas_numericas_base, 2): # Mejor un paquete para las iteraciones :)

    nombre_col_interaccion = f"{col1}_x_{col2}" 
    df_entrenamiento_interacciones[nombre_col_interaccion] = df_entrenamiento_interacciones[col1] * df_entrenamiento_interacciones[col2]
    
    df_prueba_interacciones[nombre_col_interaccion] = df_prueba_interacciones[col1] * df_prueba_interacciones[col2]

# Probar modelos:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score

import xgboost as xgb

In [ ]:
# --- Bucle de Entrenamiento y Evaluación de Modelos ---
print("\n--- Iniciando Evaluación de Modelos ---")
warnings.filterwarnings('ignore')

conjuntos_datos_a_evaluar = []

conjuntos_datos_a_evaluar.append({'nombre': 'Base', 'datos': df_entrenamiento_base})
conjuntos_datos_a_evaluar.append({'nombre': 'Caracteristicas Reducidas', 'datos': df_entrenamiento_reducido})
conjuntos_datos_a_evaluar.append({'nombre': 'Interaccion de Caracteristicas', 'datos': df_entrenamiento_interacciones})


# Preparar lista de modelos
modelos_a_evaluar = [
    #('Árbol de Decisión', DecisionTreeClassifier(random_state=42, max_depth=500)), # Traducido
    #('Naive Bayes Gaussiano', GaussianNB()), # Traducido
    #('Random Forest', RandomForestClassifier(random_state=42, n_jobs=-1)),
    #('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
    #('Bagging', BaggingClassifier(random_state=42, n_jobs=-1)),
    ('XGBoost', xgb.XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1, max_depth = 1000, device = 'gpu'))
]

# Definir StratifiedKFold y el F1 scorer
evaluador_f1 = make_scorer(f1_score, pos_label=1, zero_division=0)
num_divisiones = 4 
kfold_estratificado = StratifiedKFold(n_splits=num_divisiones, shuffle=True, random_state=42) 

resultados_completos = {} 
resumen_f1_promedio = {} 

for info_conjunto in conjuntos_datos_a_evaluar: 
    nombre_conjunto = info_conjunto['nombre'] 
    df_actual_entrenamiento = info_conjunto['datos'] 
    
    print(f"\n======================================================================")
    print(f"--- Evaluando Conjunto de Datos: {nombre_conjunto} (Forma: {df_actual_entrenamiento.shape}) ---")
    print(f"======================================================================")

    if objetivo not in df_actual_entrenamiento.columns:
        print(f"  ERROR: La columna objetivo '{objetivo}' no se encuentra en '{nombre_conjunto}'. Omitiendo.")
        continue

    X_entrenamiento = df_actual_entrenamiento.drop(columns=[objetivo]) 
    y_entrenamiento = df_actual_entrenamiento[objetivo] 
    
    
    resumen_f1_promedio[nombre_conjunto] = {}
    resultados_completos[nombre_conjunto] = {}

    for nombre_modelo, instancia_modelo in modelos_a_evaluar: 
        print(f"\n  --- Modelo: {nombre_modelo} ---")
        try:
            puntuaciones_cv_f1 = cross_val_score(instancia_modelo, X_entrenamiento, y_entrenamiento, cv=kfold_estratificado, scoring=evaluador_f1, n_jobs=-1) 
            
            f1_promedio_modelo = np.mean(puntuaciones_cv_f1) 
            desv_est_f1_modelo = np.std(puntuaciones_cv_f1) 
            
            resumen_f1_promedio[nombre_conjunto][nombre_modelo] = f1_promedio_modelo
            resultados_completos[nombre_conjunto][nombre_modelo] = {'f1_promedio': f1_promedio_modelo, 'desv_est_f1': desv_est_f1_modelo, 'f1_por_fold': puntuaciones_cv_f1.tolist()}
            
            print(f"    F1-Score Promedio ({num_divisiones} folds): {f1_promedio_modelo:.4f} (std: {desv_est_f1_modelo:.4f})")
            print(f"    F1-Scores por fold: {[round(s, 4) for s in puntuaciones_cv_f1]}")
            
        except Exception as e:
            print(f"    ERROR al entrenar/evaluar {nombre_modelo} en {nombre_conjunto}: {e}")
            resumen_f1_promedio[nombre_conjunto][nombre_modelo] = np.nan
            resultados_completos[nombre_conjunto][nombre_modelo] = {'f1_promedio': np.nan, 'desv_est_f1': np.nan, 'f1_por_fold': [], 'error': str(e)}

# Imprimir tabla resumen
print(f"\n\n======================================================================")
print(f"--- Tabla Resumen de F1-Score Promedio (clase positiva=1) ---")
print(f"======================================================================")


df_resumen = pd.DataFrame(resumen_f1_promedio).T 
orden_modelos = [name for name, _ in modelos_a_evaluar] 
columnas_existentes_modelos = [m for m in orden_modelos if m in df_resumen.columns] 
if columnas_existentes_modelos:
    df_resumen = df_resumen[columnas_existentes_modelos]

print(df_resumen.to_string(float_format="%.4f"))


--- Iniciando Evaluación de Modelos ---

--- Evaluando Conjunto de Datos: Base (Forma: (15000, 12)) ---

  --- Modelo: Proceso Gaussiano ---


Exception ignored in: <function ResourceTracker.__del__ at 0x10d2cc540>
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x10d0f8540>
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/l

KeyboardInterrupt: 

## Funcion Predict Modelo
Esta funcion, dado un modelo y una df hace un predict y lo guarda en su correspondiente csv

In [70]:
def predictCSV(model, model_name, df_toPredict, df_name):
    columns = ['id', 'fallo']
    csv_file = f"CSVModelos/{model_name}_{df_name}.csv"
    header = True

    for i in range(0, len(df_toPredict), 500):
        test_predict = model.predict(df_toPredict[i:i+500])
        
        indi = [j for j in range(i, i+500,1)]
        valor = [test_predict[k] for k in range(500)]

        results = pd.DataFrame(list(zip(indi,valor)), columns=columns)

        results.to_csv(csv_file, mode='a', header=header, index=False)
        header = False

## DECISIONTREE


In [71]:
def predictArbolDecision():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"ArbolDecision",df_toTest,"Reducido")

## NAIVEBAYES


In [72]:
def predictNaiveBayes():
    df_to_train = df_entrenamiento_base.copy()
    df_toTest = df_prueba_base.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = GaussianNB()
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"NaiveBayes",df_toTest,"Basico")


## RANDOMFOREST


In [73]:
def predictRandomForest():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = RandomForestClassifier(random_state=42, n_jobs=-1)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"RandomForest",df_toTest,"Reducido")

## GRADIENTBOOSTING

In [74]:

def predictGradientBoosting():
    df_to_train = df_entrenamiento_base.copy()
    df_toTest = df_prueba_base.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = GradientBoostingClassifier(random_state=42)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"GradientBoosting",df_toTest,"Basico")


## BAGGING

In [ ]:
def predictBagging():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = BaggingClassifier(random_state=42, n_jobs=-1)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"ArbolDecision",df_toTest,"Reducido")

## XGBoost

In [86]:
def predictXGBoost():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1, max_depth = 500, device = 'gpu')
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"XGBoost",df_toTest,"Reducido")

In [ ]:
#predictArbolDecision()
#predictNaiveBayes()
#predictRandomForest()
#predictGradientBoosting()
#predictBagging()
#predictXGBoost()

Exception ignored in: <function ResourceTracker.__del__ at 0x10d2d8540>
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x10b880540>
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/usr/local/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/l